# --- `Day 23`: Amphipod ---

In [1]:
import aocd
import re
import heapq
import operator
from collections import Counter, defaultdict, deque
from itertools import combinations
from functools import reduce, lru_cache

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def count(iterable, predicate = bool):
    return sum([1 for item in iterable if predicate(item)])

def first(iterable, default = None):
    return next(iter(iterable), default)

def lmap(func, *iterables):
    return list(map(func, *iterables))

def ints(s):
    return lmap(int, re.findall(r"-?\d+", s))

def words(s):
    return re.findall(r"[a-zA-Z]+", s)

def list_diff(x):
    return [b - a for a, b in zip(x, x[1:])]

def binary_to_int(lst):
    return int("".join(str(i) for i in lst), 2)

def get_column(lst, index):
    return [x[index] for x in lst]

In [51]:
def parse_line(line): 
    return str(line)
    
def parse_input(input):
    return list(map(parse_line, input.splitlines()))

In [52]:
final_input = parse_input(aocd.get_data(day=23, year=2021))
print(final_input[:5])

['#############', '#...........#', '###C#A#B#D###', '  #D#C#A#B#', '  #########']


In [53]:
test_input = parse_input('''\
#############
#...........#
###B#C#B#D###
  #D#C#B#A#
  #D#B#A#C#
  #A#D#C#A#
  #########
''')

print(test_input)

['#############', '#...........#', '###B#C#B#D###', '  #D#C#B#A#', '  #D#B#A#C#', '  #A#D#C#A#', '  #########']


### Helpers

In [46]:
def printBoard(board):
    print("")
    hallway, cave = board
    print("#############")
    print(f"#{''.join(hallway)}#")
    print(f"###{cave[0][0]}#{cave[1][0]}#{cave[2][0]}#{cave[3][0]}###")
    print(f"  #{cave[0][1]}#{cave[1][1]}#{cave[2][1]}#{cave[3][1]}#  ")
    print(f"  #{cave[0][2]}#{cave[1][2]}#{cave[2][2]}#{cave[3][2]}#  ")
    print(f"  #{cave[0][3]}#{cave[1][3]}#{cave[2][3]}#{cave[3][3]}#  ")
    print("  #########")
    
def isCaveOpen(board, piece):
    _, cave = board
    if piece == 'A':
        if cave[0][0] == '.' and cave[0][1] == '.' and cave[0][2] == '.' and cave[0][3] == '.':
            return 3
        elif cave[0][0] == '.' and cave[0][1] == '.' and cave[0][2] == '.' and cave[0][3] == "A":
            return 2
        elif cave[0][0] == '.' and cave[0][1] == '.' and cave[0][2] == 'A' and cave[0][3] == "A":
            return 1
        elif cave[0][0] == '.' and cave[0][1] == 'A' and cave[0][2] == 'A' and cave[0][3] == "A":
            return 0
    elif piece == 'B':
        if cave[1][0] == '.' and cave[1][1] == '.' and cave[1][2] == '.' and cave[1][3] == '.':
            return 3
        elif cave[1][0] == '.' and cave[1][1] == '.' and cave[1][2] == '.' and cave[1][3] == "B":
            return 2
        elif cave[1][0] == '.' and cave[1][1] == '.' and cave[1][2] == 'B' and cave[1][3] == "B":
            return 1
        elif cave[1][0] == '.' and cave[1][1] == 'B' and cave[1][2] == 'B' and cave[1][3] == "B":
            return 0
    elif piece == 'C':
        if cave[2][0] == '.' and cave[2][1] == '.' and cave[2][2] == '.' and cave[2][3] == '.':
            return 3
        elif cave[2][0] == '.' and cave[2][1] == '.' and cave[2][2] == '.' and cave[2][3] == "C":
            return 2
        elif cave[2][0] == '.' and cave[2][1] == '.' and cave[2][2] == 'C' and cave[2][3] == "C":
            return 1
        elif cave[2][0] == '.' and cave[2][1] == 'C' and cave[2][2] == 'C' and cave[2][3] == "C":
            return 0
    elif piece == 'D':
        if cave[3][0] == '.' and cave[3][1] == '.' and cave[3][2] == '.' and cave[3][3] == '.':
            return 3
        elif cave[3][0] == '.' and cave[3][1] == '.' and cave[3][2] == '.' and cave[3][3] == "D":
            return 2
        elif cave[3][0] == '.' and cave[3][1] == '.' and cave[3][2] == 'D' and cave[3][3] == "D":
            return 1
        elif cave[3][0] == '.' and cave[3][1] == 'D' and cave[3][2] == 'D' and cave[3][3] == "D":
            return 0
    return None

def get00Moves(board, source, piece, spaces):
    hallway, cave = board
    moves = []
    caveOpen = isCaveOpen(board, piece)
    if hallway[1] == '.':
        moves.append((piece, source, 1, 2 + spaces))
        if hallway[0] == '.':
            moves.append((piece, source, 0, 3 + spaces))
    if hallway[3] == '.':
        moves.append((piece, source, 3, 2 + spaces))
        if piece == 'B' and caveOpen != None:
            moves.append((piece, source, 15 + caveOpen, 4 + spaces + caveOpen))
        if hallway[5] == '.':
            moves.append((piece, source, 5, 4 + spaces))
            if piece == 'C' and caveOpen != None:
                moves.append((piece, source, 19 + caveOpen, 6 + spaces + caveOpen))
            if hallway[7] == '.':
                moves.append((piece, source, 7, 6 + spaces))
                if piece == 'D' and caveOpen != None:
                    moves.append((piece, source, 23 + caveOpen, 8 + spaces + caveOpen))
                if hallway[9] == '.':
                    moves.append((piece, source, 9, 8 + spaces))
                    if hallway[10] == '.':
                        moves.append((piece, source, 10, 9 + spaces))
    return moves

def get10Moves(board, source, piece, spaces):
    hallway, cave = board
    moves = []
    caveOpen = isCaveOpen(board, piece)
    if hallway[3] == '.':
        moves.append((piece, source, 3, 2 + spaces))
        if piece == 'A' and caveOpen != None:
            moves.append((piece, source, 11 + caveOpen, 4 + spaces + caveOpen))
        if hallway[1] == '.':
            moves.append((piece, source, 1, 4 + spaces))
            if hallway[0] == '.':
                moves.append((piece, source, 0, 5 + spaces))
    if hallway[5] == '.':
        moves.append((piece, source, 5, 2 + spaces))
        if piece == 'C' and caveOpen != None:
            moves.append((piece, source, 19 + caveOpen, 4 + spaces + caveOpen))
        if hallway[7] == '.':
            moves.append((piece, source, 7, 4 + spaces))
            if piece == 'D' and caveOpen != None:
                moves.append((piece, source, 23 + caveOpen, 6 + spaces + caveOpen))
            if hallway[9] == '.':
                moves.append((piece, source, 9, 6 + spaces))
                if hallway[10] == '.':
                    moves.append((piece, source, 10, 7 + spaces))
    return moves

def get20Moves(board, source, piece, spaces):
    hallway, cave = board
    moves = []
    caveOpen = isCaveOpen(board, piece)
    if hallway[5] == '.':
        moves.append((piece, source, 5, 2 + spaces))
        if piece == 'B' and caveOpen != None:
            moves.append((piece, source, 15 + caveOpen, 4 + spaces + caveOpen))
        if hallway[3] == '.':
            moves.append((piece, source, 3, 4 + spaces))
            if piece == 'A' and caveOpen != None:
                moves.append((piece, source, 11 + caveOpen, 6 + spaces + caveOpen))
            if hallway[1] == '.':
                moves.append((piece, source, 1, 6 + spaces))
                if hallway[0] == '.':
                    moves.append((piece, source, 0, 7 + spaces))
    if hallway[7] == '.':
        moves.append((piece, source, 7, 2 + spaces))
        if piece == 'D' and caveOpen != None:
            moves.append((piece, source, 23 + caveOpen, 4 + spaces + caveOpen))
        if hallway[9] == '.':
            moves.append((piece, source, 9, 4 + spaces))
            if hallway[10] == '.':
                moves.append((piece, source, 10, 5 + spaces))
    return moves

def get30Moves(board, source, piece, spaces):
    hallway, cave = board
    moves = []
    caveOpen = isCaveOpen(board, piece)
    if hallway[7] == '.':
        moves.append((piece, source, 7, 2 + spaces))
        if piece == 'C' and caveOpen != None:
            moves.append((piece, source, 19 + caveOpen, 4 + spaces + caveOpen))  
        if hallway[5] == '.':
            moves.append((piece, source, 5, 4 + spaces))
            if piece == 'B' and caveOpen != None:
                moves.append((piece, source, 15 + caveOpen, 6 + spaces + caveOpen))
            if hallway[3] == '.':
                moves.append((piece, source, 3, 6 + spaces))
                if piece == 'A' and caveOpen != None:
                    moves.append((piece, source, 11 + caveOpen, 8 + spaces + caveOpen))
                if hallway[1] == '.':
                    moves.append((piece, source, 1, 8 + spaces))
                    if hallway[0] == '.':
                        moves.append((piece, source, 0, 9 + spaces))
    
    if hallway[9] == '.':
        moves.append((piece, source, 9, 2 + spaces))
        if hallway[10] == '.':
            moves.append((piece, source, 10, 3 + spaces))
    return moves

def getMoves(board):
    hallway, cave = board
    moves = []
    
    # from hallway
    places = {'A':11, 'B':15, 'C':19, 'D':23}
    for i,n in enumerate(hallway):
        if n != '.':
            caveOpen = isCaveOpen(board, n)
            if caveOpen != None:
                times = None
                if n == 'A':
                    if i == 0 and hallway[1] == '.':
                        times = 3
                    elif i == 1 or i == 3:
                        times = 2
                    elif i == 5 and hallway[3] == '.':
                        times = 4
                    elif i == 7 and hallway[3] == '.' and hallway[5] == '.':
                        times = 6
                    elif i == 9 and hallway[3] == '.' and hallway[5] == '.' and hallway[7] == '.':
                        times = 8
                    elif i == 10 and hallway[3] == '.' and hallway[5] == '.' and hallway[7] == '.' and hallway[9] == '.':
                        times = 9
                elif n == 'B':
                    if i == 0 and hallway[1] == '.' and hallway[3] == '.':
                        times = 5
                    elif i == 1 and hallway[3] == '.':
                        times = 4
                    elif i == 3 or i == 5:
                        times = 2
                    elif i == 7 and hallway[5] == '.':
                        times = 4
                    elif i == 9 and hallway[5] == '.' and hallway[7] == '.':
                        times = 6
                    elif i == 10 and hallway[5] == '.' and hallway[7] == '.' and hallway[9] == '.':
                        times = 7
                elif n == 'C':
                    if i == 0 and hallway[1] == '.' and hallway[3] == '.' and hallway[5] == '.':
                        times = 7
                    elif i == 1 and hallway[3] == '.' and hallway[5] == '.':
                        times = 6
                    elif i == 3 and hallway[5] == '.':
                        times = 4
                    elif i == 5 or i == 7:
                        times = 2
                    elif i == 9 and hallway[7] == '.':
                        times = 4
                    elif i == 10 and hallway[7] == '.' and hallway[9] == '.':
                        times = 5
                elif n == 'D':
                    if i == 0 and hallway[1] == '.' and hallway[3] == '.' and hallway[5] == '.' and hallway[7] == '.':
                        times = 9
                    elif i == 1 and hallway[3] == '.' and hallway[5] == '.' and hallway[7] == '.':
                        times = 8
                    elif i == 3 and hallway[5] == '.' and hallway[7] == '.':
                        times = 6
                    elif i == 5 and hallway[7] == '.':
                        times = 4
                    elif i == 7 or i == 9:
                        times = 2
                    elif i == 10 and hallway[9] == '.':
                        times = 3
                if times != None:
                    moves.append((n, i, places[n] + caveOpen, times + caveOpen))

    # from cave
    if cave[0][0] in ['B','C','D'] or cave[0][1] in ['B','C','D'] or cave[0][2] in ['B','C','D'] or cave[0][3] in ['B','C','D']:
        if cave[0][0] != '.':
            moves.extend(get00Moves(board, 11, cave[0][0], 0))
        elif cave[0][1] != '.':
            moves.extend(get00Moves(board, 12, cave[0][1], 1))
        elif cave[0][2] != '.':
            moves.extend(get00Moves(board, 13, cave[0][2], 2))
        elif cave[0][3] in ['B', 'C', 'D']:
            moves.extend(get00Moves(board, 14, cave[0][3], 3))
    if cave[1][0] in ['A','C','D'] or cave[1][1] in ['A','C','D'] or cave[1][2] in ['A','C','D'] or cave[1][3] in ['A','C','D']:
        if cave[1][0] != '.':
            moves.extend(get10Moves(board, 15, cave[1][0], 0))
        elif cave[1][1] != '.':
            moves.extend(get10Moves(board, 16, cave[1][1], 1))
        elif cave[1][2] != '.':
            moves.extend(get10Moves(board, 17, cave[1][2], 2))
        elif cave[1][3] in ['A', 'C', 'D']:
            moves.extend(get10Moves(board, 18, cave[1][3], 3))
    if cave[2][0] in ['A','B','D'] or cave[2][1] in ['A','B','D'] or cave[2][2] in ['A','B','D'] or cave[2][3] in ['A','B','D']:
        if cave[2][0] != '.':
            moves.extend(get20Moves(board, 19, cave[2][0], 0))
        elif cave[2][1] != '.':
            moves.extend(get20Moves(board, 20, cave[2][1], 1))
        elif cave[2][2] != '.':
            moves.extend(get20Moves(board, 21, cave[2][2], 2))
        elif cave[2][3] in ['A', 'B', 'D']:
            moves.extend(get20Moves(board, 22, cave[2][3], 3))
    if cave[3][0] in ['A','B','C'] or cave[3][1] in ['A','B','C'] or cave[3][2] in ['A','B','C'] or cave[3][3] in ['A','B','C']:
        if cave[3][0] != '.':
            moves.extend(get30Moves(board, 23, cave[3][0], 0))
        elif cave[3][1] != '.':
            moves.extend(get30Moves(board, 24, cave[3][1], 1))
        elif cave[3][2] != '.':
            moves.extend(get30Moves(board, 25, cave[3][2], 2))
        elif cave[3][3] in ['A', 'B', 'C']:
            moves.extend(get30Moves(board, 26, cave[3][3], 3))
    return moves

## Solution 1

In [55]:
def checkWin(board):
    hallway, cave = board
    if cave[0][0] == cave[0][1] == cave[0][2] == cave[0][3] == 'A':
        if cave[1][0] == cave[1][1] == cave[1][2] == cave[1][3] == 'B':
            if cave[2][0] == cave[2][1] == cave[2][2] == cave[2][3] == 'C':
                if cave[3][0] == cave[3][1] == cave[3][2] == cave[3][3] == 'D':
                    return True
    return False
    
def updateCave(caves, piece, source, dest):
    newCaves = [[a,b,c,d] for a,b,c,d in caves]
    if source == 11:
        newCaves[0][0] = '.'
    elif source == 12:
        newCaves[0][1] = '.'
    elif source == 13:
        newCaves[0][2] = '.'
    elif source == 14:
        newCaves[0][3] = '.' 
    elif source == 15:
        newCaves[1][0] = '.'
    elif source == 16:
        newCaves[1][1] = '.'
    elif source == 17:
        newCaves[1][2] = '.'
    elif source == 18:
        newCaves[1][3] = '.'
    elif source == 19:
        newCaves[2][0] = '.'
    elif source == 20:
        newCaves[2][1] = '.'
    elif source == 21:
        newCaves[2][2] = '.'
    elif source == 22:
        newCaves[2][3] = '.'      
    elif source == 23:
        newCaves[3][0] = '.'
    elif source == 24:
        newCaves[3][1] = '.'
    elif source == 25:
        newCaves[3][2] = '.'
    elif source == 26:
        newCaves[3][3] = '.'
        
    if dest == 11:
        newCaves[0][0] = piece
    elif dest == 12:
        newCaves[0][1] = piece
    elif dest == 13:
        newCaves[0][2] = piece
    elif dest == 14:
        newCaves[0][3] = piece       
    elif dest == 15:
        newCaves[1][0] = piece
    elif dest == 16:
        newCaves[1][1] = piece
    elif dest == 17:
        newCaves[1][2] = piece
    elif dest == 18:
        newCaves[1][3] = piece
    elif dest == 19:
        newCaves[2][0] = piece
    elif dest == 20:
        newCaves[2][1] = piece
    elif dest == 21:
        newCaves[2][2] = piece
    elif dest == 22:
        newCaves[2][3] = piece
    elif dest == 23:
        newCaves[3][0] = piece
    elif dest == 24:
        newCaves[3][1] = piece
    elif dest == 25:
        newCaves[3][2] = piece
    elif dest == 26:
        newCaves[3][3] = piece
        
    return tuple([(a,b,c,d) for a,b,c,d in newCaves])
    
@lru_cache(None)
def move(board, layer = 0):
    hallway, cave = board
    costs = {'A':1, 'B':10, 'C':100, 'D':1000}
    
    if checkWin(board):
        return 0
    
    moves = getMoves(board)
    if len(moves) == 0:
        return None
    
    best = None
    for piece, source, dest, spaces in moves:
        newBoard = tuple()
        if dest <= 10:
            newHallway = [x for x in hallway]
            newHallway[dest] = piece
            newHallway = tuple(newHallway)
            newCave = updateCave(cave, piece, source, dest)
            newBoard = (newHallway, newCave)
        elif source <= 10:
            newHallway = [x for x in hallway]
            newHallway[source] = '.'
            newHallway = tuple(newHallway)
            newCave = updateCave(cave, piece, source, dest)
            newBoard = (newHallway, newCave)
        else:
            newCave = updateCave(cave, piece, source, dest)
            newBoard = (hallway, newCave)
        
        #printBoard(newBoard)
        temp = move(newBoard, layer + 1)
        if temp != None:
            cost = costs[piece] * spaces
            temp += cost
            if best == None or temp < best:
                best = temp
    if best == None:
        return None
    return best  

def solve_1():
    hallway = ['.'] * 11
    hallway = tuple(hallway)
    #caves = ((".", ".", "D", "A"), (".", ".", ".", "D"), (".", "C", "C", "C"), ("D", "A", "C", "A"))

    #caves = (("B", "D", "D", "A"), ("C", "C", "B", "D"), ("B", "B", "A", "C"), ("D", "A", "C", "A"))
    caves = (("C", "D", "D", "D"), ("A", "C", "B", "C"), ("B", "B", "A", "A"), ("D", "A", "C", "B"))
    board = (hallway, caves)
    #printBoard(board)
    cost = move(board)
    return cost
           
solve_1()

51436